## **Import Libraries** ##

In [15]:
# Importing req. libraries
import os
import numpy as np
from PIL import Image
from skimage.feature import hog
from skimage import exposure
import cv2

## **Convert Img to Numpy Array** ##

In [16]:
# Function to load image data and convert to numpy array
def convertImgToNumpyArr(image_path):
    try:
        img = Image.open(image_path)  # creating reference variable img to access image data
        img = img.resize((100, 100))  # resize image to 100x100 in case it is not
        img = np.array(img)  # converting image data from JPG to numpy array
        #img = img / 255.0  # normalizing RGB values (cv2 needs 0-255 range)
        return img
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")  # error handling
        return None

## **Compute HOG Features** ##

In [22]:
# Function to compute HOG features of an image
def computeHOGFeatures(image):
    
    # conversion to grayscale
    grayImg = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    grayImg = grayImg / 255.0  # normalizing

    # computing HOG features
    hogFeatures, hogImg = hog(grayImg, pixels_per_cell=(16,16), orientations=9, cells_per_block=(2, 2), visualize=True) # 9 orientations, 8x8 pixels per cell, 2x2 cells per block

    # increasing the contrast of the image
    hogImg = exposure.rescale_intensity(hogImg, in_range=(0, 10))

    return hogFeatures, hogImg

## **Compute Colour Histogram Features** ##

In [23]:
# Function to compute colour histogram features for an image

def computeColourHist(image, bins=20):

    # conversion from RGB to HSV
    hsvImg = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    hsvImg = hsvImg / 255.0  # normalizing
    
    # extracting histograms for Hue, Saturation, and Value
    hueHist = np.histogram(hsvImg[:,:,0], bins=bins, range=(0, 1))[0]  
    satHist = np.histogram(hsvImg[:,:,1], bins=bins, range=(0, 1))[0]  
    valHist = np.histogram(hsvImg[:,:,2], bins=bins, range=(0, 1))[0]  
    
    # concatenating the histograms of Hue, Saturation, and Value and returning
    return np.concatenate((hueHist, satHist, valHist))

## **Extracting Features** ##

In [ ]:
# Function to extract features and save them as numpy arrays
def saveImgFeaturesAsNumpyArr(src_path, save_path):
    print(f"Original Image Folder: {src_path}")
    os.makedirs(os.path.dirname(save_path), exist_ok=True)  # creating the save folder if it doesn't exist
    print(f"Saving Image Features Folder: {save_path}")
    
    # going through each fruit folder in the source path (train or test)
    for fruit_folder in os.listdir(src_path):
        fruit_folder_path = os.path.join(src_path, fruit_folder)
        
        if os.path.isdir(fruit_folder_path):  # checking if the path actually points to a folder
            print(f"Currently Processing Fruit Images of: {fruit_folder}")
            
            # going through each image file in the current folder
            for img in os.listdir(fruit_folder_path):
                img_path = os.path.join(fruit_folder_path, img)
                
                if img.endswith('.jpg'):  # checking if the file is an jpg image
                    img_as_numpy_arr = convertImgToNumpyArr(img_path)
                    if img_as_numpy_arr is None:  # handling image processing error
                        print(f"Error processing image {img_path}")
                    else:
                        # computing the feature vector (HOG + Colour Histogram)
                        hogFeatures, _ = computeHOGFeatures(img_as_numpy_arr)
                        colourHistFeatures = computeColourHist(img_as_numpy_arr)
                        # print("HOG features shape:", hogFeatures.shape)
                        # print("Colour Histogram features shape:", colourHistFeatures.shape)

                        # joining both feature sets
                        featureVector = np.concatenate((hogFeatures, colourHistFeatures))
                        featureVector = featureVector.astype(np.float32) # converting to float32 for saving storage space
                        # print("Feature vector length:", len(featureVector))

                        # creating the respective save path for each feature vector
                        subfolder_path = os.path.relpath(fruit_folder_path, src_path)
                        save_file_name = f"{os.path.splitext(img)[0]}.npy"  # saving file with .npy extension
                        img_save_path = os.path.join(save_path, subfolder_path, save_file_name)
                        
                        # creating the save folder if it doesn't exist
                        os.makedirs(os.path.dirname(img_save_path), exist_ok=True)
                        np.save(img_save_path, featureVector)

## **Image Conversion to Features** ##

In [ ]:
# Paths to the train and test folders
train_path = '../data/fruits-360/Training'
test_path = '../data/fruits-360/Test'

# Training Images to Features:
print("Processing Training Images")
saveImgFeaturesAsNumpyArr(train_path, 'imgFeatures/Training')

# Testing Images to Features:
print("Processing Testing Images")
saveImgFeaturesAsNumpyArr(test_path, 'imgFeatures/Testing')